### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Liver_cirrhosis/GSE85550'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

is_gene_available = True  # Assuming gene expression data is available based on typical GEO series for liver fibrosis

# Analyze Sample Characteristics Dictionary
sample_characteristics = {
    0: ['patient: HUc034', 'patient: HUc035', 'patient: HUc036', 'patient: HUc037', 'patient: HUc038', 'patient: HUc039', 'patient: HUc041', 'patient: HUc042', 'patient: HUc043', 'patient: HUc044', 'patient: HUc045', 'patient: HUc046', 'patient: HUc047', 'patient: HUc048', 'patient: HUc049', 'patient: HUc050', 'patient: HUc051', 'patient: HUc052', 'patient: HUc053', 'patient: HUc054', 'patient: HUc055', 'patient: HUc056', 'patient: HUc057', 'patient: HUc058', 'patient: HUc059', 'patient: HUc060', 'patient: HUc061', 'patient: HUc062', 'patient: HUc063', 'patient: HUc064'], 
    1: ['tissue: liver biopsy'], 
    2: ['time_point: Baseline', 'time_point: Follow-up']
}

# Heuristic to identify key rows based on sample characteristics
trait_row = 2 if 'time_point: Baseline' in sample_characteristics[2] else None
age_row = None  # No information available for age
gender_row = None  # No information available for gender

# Define conversion functions
def convert_trait(value):
    return 1 if 'Follow-up' in value else 0

# Since age and gender are not available, no conversion functions are defined
convert_age = lambda value: None
convert_gender = lambda value: None

# Save Metadata
save_cohort_info('GSE85550', './preprocessed/Liver_cirrhosis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Liver_cirrhosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Liver_cirrhosis/trait_data/GSE85550.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# Since requires_gene_mapping = False, we must skip this step.


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Liver_cirrhosis/gene_data/GSE85550.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Liver_cirrhosis')

# 4. Save the cohort information with the corrected variable name.
save_cohort_info('GSE85550', './preprocessed/Liver_cirrhosis/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Liver_cirrhosis/GSE85550.csv'
    unbiased_merged_data.to_csv(csv_path)
